In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
import warnings
warnings.filterwarnings('ignore')


## model.py

In [4]:
import torch
from torch import nn
from torch.nn import Parameter
import torch.nn.functional as F



def l2normalize(v, eps=1e-12):
    return v / (v.norm() + eps)


class SpectralNorm(nn.Module):
    def __init__(self, module, name='weight', power_iterations=1):
        super(SpectralNorm, self).__init__()
        self.module = module
        self.name = name
        self.power_iterations = power_iterations
        if not self._made_params():
            self._make_params()

    def _update_u_v(self):
        u = getattr(self.module, self.name + "_u")
        v = getattr(self.module, self.name + "_v")
        w = getattr(self.module, self.name + "_bar")

        height = w.data.shape[0]
        for _ in range(self.power_iterations):
            v.data = l2normalize(torch.mv(torch.t(w.view(height,-1).data), u.data))
            u.data = l2normalize(torch.mv(w.view(height,-1).data, v.data))

        # sigma = torch.dot(u.data, torch.mv(w.view(height,-1).data, v.data))
        sigma = u.dot(w.view(height, -1).mv(v))
        setattr(self.module, self.name, w / sigma.expand_as(w))

    def _made_params(self):
        try:
            u = getattr(self.module, self.name + "_u")
            v = getattr(self.module, self.name + "_v")
            w = getattr(self.module, self.name + "_bar")
            return True
        except AttributeError:
            return False

    def _make_params(self):
        w = getattr(self.module, self.name)

        height = w.data.shape[0]
        width = w.view(height, -1).data.shape[1]

        u = Parameter(w.data.new(height).normal_(0, 1), requires_grad=False)
        v = Parameter(w.data.new(width).normal_(0, 1), requires_grad=False)
        u.data = l2normalize(u.data)
        v.data = l2normalize(v.data)
        w_bar = Parameter(w.data)

        del self.module._parameters[self.name]

        self.module.register_parameter(self.name + "_u", u)
        self.module.register_parameter(self.name + "_v", v)
        self.module.register_parameter(self.name + "_bar", w_bar)

    def forward(self, *args):
        self._update_u_v()
        return self.module.forward(*args)



def upconv(in_channels, out_channels, kernel_size, stride=2, padding=2, batch_norm=True, spectral_norm=False):
    """Creates a upsample-and-convolution layer, with optional batch normalization.
    """
    layers = []
    if stride>1:
        layers.append(nn.Upsample(scale_factor=stride))
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=1, padding=padding, bias=False)
    if spectral_norm:
        layers.append(SpectralNorm(conv_layer))
    else:
        layers.append(conv_layer)
    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)


def conv(in_channels, out_channels, kernel_size, stride=2, padding=2, batch_norm=True, init_zero_weights=False, spectral_norm=False):
    """Creates a convolutional layer, with optional batch normalization.
    """
    layers = []
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False)
    if init_zero_weights:
        conv_layer.weight.data = torch.randn(out_channels, in_channels, kernel_size, kernel_size) * 0.001

    if spectral_norm:
        layers.append(SpectralNorm(conv_layer))
    else:
        layers.append(conv_layer)

    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)


class ResnetBlock(nn.Module):
    def __init__(self, conv_dim):
        super(ResnetBlock, self).__init__()
        self.conv_layer = conv(in_channels=conv_dim, out_channels=conv_dim, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        out = x + self.conv_layer(x)
        return out



class DCGenerator(nn.Module):
    def __init__(self, noise_size, conv_dim, spectral_norm=False):
        super(DCGenerator, self).__init__()

        self.conv_dim = conv_dim

        self.linear_bn = nn.Sequential(nn.Linear(in_features=noise_size, out_features=conv_dim*4*4*4), nn.Flatten())

        self.upconv1 =  upconv(in_channels=conv_dim*4, out_channels=conv_dim*2,
                              kernel_size=5, stride=2, padding=2, spectral_norm=spectral_norm)
        self.upconv2 = upconv(in_channels=conv_dim*2, out_channels=conv_dim,
                              kernel_size=5, stride=2, padding=2, spectral_norm=spectral_norm)
        self.upconv3 = upconv(in_channels=conv_dim, out_channels=3,
                              kernel_size=5, stride=2, padding=2, batch_norm=False, spectral_norm=spectral_norm)

    def forward(self, z):
        """Generates an image given a sample of random noise.

            Input
            -----
                z: BS x noise_size x 1 x 1   -->  BSx100x1x1 (during training)

            Output
            ------
                out: BS x channels x image_width x image_height  -->  BSx3x32x32 (during training)
        """
        batch_size = z.size(0)

        z = z.view(-1, 100)
        out = F.relu(self.linear_bn(z)).view(-1, self.conv_dim*4, 4, 4)    # BS x 128 x 4 x 4
        out = F.relu(self.upconv1(out))  # BS x 64 x 8 x 8
        out = F.relu(self.upconv2(out))  # BS x 32 x 16 x 16
        out = F.tanh(self.upconv3(out))  # BS x 3 x 32 x 32

        out_size = out.size()
        if out_size != torch.Size([batch_size, 3, 32, 32]):
            raise ValueError("expect {} x 3 x 32 x 32, but get {}".format(batch_size, out_size))
        return out


class DCDiscriminator(nn.Module):

    def __init__(self, conv_dim=64, spectral_norm=False):
        super(DCDiscriminator, self).__init__()

        self.conv1 = conv(in_channels=3, out_channels=conv_dim, kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.conv2 = conv(in_channels=conv_dim, out_channels=conv_dim*2, kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.conv3 = conv(in_channels=conv_dim*2, out_channels=conv_dim*4, kernel_size=5, stride=2, spectral_norm=spectral_norm)
        self.conv4 = conv(in_channels=conv_dim*4, out_channels=1, kernel_size=5, stride=2, padding=1, batch_norm=False, spectral_norm=spectral_norm)

    def forward(self, x):
        batch_size = x.size(0)

        out = F.relu(self.conv1(x))    # BS x 64 x 16 x 16
        out = F.relu(self.conv2(out))    # BS x 64 x 8 x 8
        out = F.relu(self.conv3(out))    # BS x 64 x 4 x 4
        out = self.conv4(out).squeeze()

        out_size = out.size()
        if out_size != torch.Size([batch_size,]):
            raise ValueError("expect {} x 1, but get {}".format(batch_size, out_size))
        return out


## utils.py

In [5]:
import os
from torch.autograd import Variable
import torch
#from model import DCDiscriminator, DCGenerator




def to_var(tensor, cuda=True):
    """Wraps a Tensor in a Variable, optionally placing it on the GPU.

        Arguments:
            tensor: A Tensor object.
            cuda: A boolean flag indicating whether to use the GPU.

        Returns:
            A Variable object, on the GPU if cuda==True.
    """
    if cuda:
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)


def to_data(x):
    """Converts variable to numpy."""
    if torch.cuda.is_available():
        x = x.cpu()
    return x.data.numpy()


def create_dir(directory):
    """Creates a directory if it doesn't already exist.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)


def gan_checkpoint(iteration, G, D, opts):
    """Saves the parameters of the generator G and discriminator D.
    """
    G_path = os.path.join(opts.checkpoint_dir, f'G_{iteration}.pkl')
    D_path = os.path.join(opts.checkpoint_dir, f'D_{iteration}.pkl')
    torch.save(G.state_dict(), G_path)
    torch.save(D.state_dict(), D_path)

def load_checkpoint(opts, iteration):
    """Loads the generator and discriminator models from checkpoints.
    """
    G_path = os.path.join(opts.load, f'G_{iteration}.pkl')
    D_path = os.path.join(opts.load, f'D_{iteration}.pkl')

    G = DCGenerator(noise_size=opts.noise_size, conv_dim=opts.g_conv_dim, spectral_norm=opts.spectral_norm)
    D = DCDiscriminator(conv_dim=opts.d_conv_dim)

    G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage))
    D.load_state_dict(torch.load(D_path, map_location=lambda storage, loc: storage))

    if torch.cuda.is_available():
        G.cuda()
        D.cuda()
        print('Models moved to GPU.')

    return G, D


def gan_save_samples(G, fixed_noise, iteration, opts):
    generated_images = G(fixed_noise)
    generated_images = to_data(generated_images)
    # save images in sample dir

def create_model(opts):
    """Builds the generators and discriminators.
    """
    ### GAN
    G = DCGenerator(noise_size=opts.noise_size, conv_dim=opts.g_conv_dim, spectral_norm=opts.spectral_norm)
    D = DCDiscriminator(conv_dim=opts.d_conv_dim, spectral_norm=opts.spectral_norm)

    if torch.cuda.is_available():
        G.cuda()
        D.cuda()
        print('Models moved to GPU.')
    return G, D

def sample_noise(batch_size, dim):
    """
    Generate a PyTorch Tensor of uniform random noise.

    Input:
    - batch_size: Integer giving the batch size of noise to generate.
    - dim: Integer giving the dimension of noise to generate.

    Output:
    - A PyTorch Tensor of shape (batch_size, dim, 1, 1) containing uniform
      random noise in the range (-1, 1).
    """
    return to_var(torch.rand(batch_size, dim) * 2 - 1).unsqueeze(2).unsqueeze(3)

## dataset.py

In [6]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms


def get_emoji_loader(train_path, test_path, batch_size, image_size):
    transform = transforms.Compose([
                    transforms.Resize(image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])

    train_dataset = datasets.ImageFolder(train_path, transform)
    test_dataset = datasets.ImageFolder(test_path, transform)

    train_dloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_dloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_dloader, test_dloader

## train.py

In [7]:
import torch.optim as optim
import torch
import matplotlib.pyplot as plt
import os
# from utils import create_dir, create_model, sample_noise, to_var, gan_save_samples, gan_checkpoint
# from dataset import get_emoji_loader


def gan_training_loop(dataloader, test_dataloader, opts):

    G, D = create_model(opts)

    g_params = G.parameters()  # Get generator parameters
    d_params = D.parameters()  # Get discriminator parameters

    g_optimizer = optim.Adam(g_params, opts.lr, [opts.beta1, opts.beta2])
    d_optimizer = optim.Adam(d_params, opts.lr * 2., [opts.beta1, opts.beta2])

    train_iter = iter(dataloader)

    test_iter = iter(test_dataloader)

    # Get some fixed data from domains X for sampling. These are images that are held
    # constant throughout training, that allow us to inspect the model's performance.
    fixed_noise = sample_noise(100, opts.noise_size)  # # 100 x noise_size x 1 x 1

    iter_per_epoch = len(train_iter)
    total_train_iters = opts.train_iters

    losses = {"iteration": [], "D_fake_loss": [], "D_real_loss": [], "G_loss": []}
    loss = torch.nn.MSELoss()

    try:
        for iteration in range(1, opts.train_iters + 1):

            # Reset data_iter for each epoch
            if iteration % iter_per_epoch == 0:
                train_iter = iter(dataloader)

            real_images, real_labels = next(train_iter)
            real_images, real_labels = to_var(real_images), to_var(real_labels).long().squeeze()

            # ones = Variable(torch.Tensor(real_images.shape[0]).float().cuda().fill_(1.0), requires_grad=False)

            for d_i in range(opts.d_train_iters):
                d_optimizer.zero_grad()

                # FILL THIS IN
                # 1. Compute the discriminator loss on real images

                D_out_real = D(real_images).type(torch.FloatTensor)
                ones = torch.ones(size=real_labels.size()).type(torch.FloatTensor)
                ones.requires_grad = False
                zeros = torch.zeros(size=real_labels.size()).type(torch.FloatTensor)
                zeros.requires_grad = False

                D_real_loss = loss(D_out_real, zeros)
                D_real_loss = D_real_loss.type(torch.FloatTensor)*(1/D_out_real.shape[0])
                real_labels = real_labels.type(torch.FloatTensor)

                # 2. Sample noise
                noise = sample_noise(batch_size=opts.batch_size, dim=opts.noise_size)

                # 3. Generate fake images from the noise
                fake_images = G(noise)

                # 4. Compute the discriminator loss on the fake images
                D_out_fake = D(fake_images).type(torch.FloatTensor)
                ones = torch.ones(size=D_out_fake.shape).type(torch.FloatTensor)
                ones.requires_grad = False
                D_fake_loss =loss(D_out_fake, ones)
                D_fake_loss = D_fake_loss.type(torch.FloatTensor)*(1/D_out_fake.shape[0])



                # 5. Compute the total discriminator loss
                D_total_loss = D_fake_loss + D_real_loss
                D_total_loss = D_total_loss.type(torch.FloatTensor)
                D_total_loss.backward()
                d_optimizer.step()


            ###########################################
            ###          TRAIN THE GENERATOR        ###
            ###########################################

            g_optimizer.zero_grad()

            # FILL THIS IN
            # 1. Sample noise
            noise = sample_noise(batch_size=opts.batch_size, dim=opts.noise_size)

            # 2. Generate fake images from the noise
            fake_images = G(noise)

            # 3. Compute the generator loss
            D_out_fake1 = D(fake_images).type(torch.FloatTensor)
            zeros = torch.zeros(size=D_out_fake1.shape).type(torch.FloatTensor)
            zeros.requires_grad = False
            G_loss = loss(D_out_fake1, zeros)
            G_loss = G_loss.type(torch.FloatTensor)*(1/D_out_fake1.shape[0])

            G_loss.backward()
            g_optimizer.step()

            # Print the log info
            if iteration % opts.log_step == 0:
                losses['iteration'].append(iteration)
                losses['D_real_loss'].append(D_real_loss.item())
                losses['D_fake_loss'].append(D_fake_loss.item())
                losses['G_loss'].append(G_loss.item())
                print('Iteration [{:4d}/{:4d}] | D_real_loss: {:6.4f} | D_fake_loss: {:6.4f} | G_loss: {:6.4f}'.format(
                    iteration, total_train_iters, D_real_loss.item(), D_fake_loss.item(), G_loss.item()))

            # Save the generated samples
            if iteration % opts.sample_every == 0:
                gan_save_samples(G, fixed_noise, iteration, opts)

            # Save the model parameters
            if iteration % opts.checkpoint_every == 0:
                gan_checkpoint(iteration, G, D, opts)

    except KeyboardInterrupt:
        print('Exiting early from training.')

    plt.figure()
    plt.plot(losses['iteration'], losses['D_real_loss'], label='D_real')
    plt.plot(losses['iteration'], losses['D_fake_loss'], label='D_fake')
    plt.plot(losses['iteration'], losses['G_loss'], label='G')
    plt.legend()
    plt.savefig(os.path.join(opts.sample_dir, f'losses_{opts.lr}_{opts.d_train_iters}_{opts.spectral_norm}_.png'))
    plt.close()




def train(opts):

    dataloader_X, test_dataloader_X = get_emoji_loader(opts.train_path, opts.test_path, opts.batch_size, opts.image_size)

    # Set the random seed manually for reproducibility.
    # ......

    create_dir(opts.checkpoint_dir)
    create_dir(opts.sample_dir)

    gan_training_loop(dataloader_X, test_dataloader_X, opts)



# if __name__ == '__main__':
#     # Load config file
#     args = {
#               'image_size':32,
#               'g_conv_dim':32,
#               'd_conv_dim':64,
#               'noise_size':100,
#               'train_iters':15000,
#               'train_path':'/content/drive/MyDrive/emojis/Apple',
#               'test_path': '/content/drive/MyDrive/emojis/Test_Apple',
#               'lr':0.0005,
#               'beta1':0.5,
#               'beta2':0.999,
#               'batch_size':64,
#               'checkpoint_dir': '/content/drive/MyDrive/emojis/checkpoints',
#               'sample_dir': '/content/drive/MyDrive/emojis/samples',
#               'load': None,
#               'log_step':200,
#               'sample_every':1000,
#               'checkpoint_every':1000,
#               'spectral_norm': False,
#               'gradient_penalty': False,
#               'd_train_iters': 1
# }
#     train(args)

In [8]:
!pip install access-dict-by-dot

  Preparing metadata (setup.py) ... done
  Created wheel for access-dict-by-dot: filename=access_dict_by_dot-0.0.4-py3-none-any.whl size=2237 sha256=91bc402d134bcd4004c4b5ec5ff32553766b68c6876f5d6fb085cec5463bf883
  Stored in directory: /root/.cache/pip/wheels/ff/e4/ac/0b2f8e9a6df7fe262ed95622941605a1bb69da435a4013db74
Successfully built access-dict-by-dot


In [ ]:
from access_dict_by_dot import AccessDictByDot

args = {
              'image_size':32,
              'g_conv_dim':32,
              'd_conv_dim':64,
              'noise_size':100,
              'train_iters':10000,
              'train_path': '/content/drive/MyDrive/Apple/',
              "test_path": '/content/drive/MyDrive/emojis/Test_Apple',
              'lr':0.0005,
              'beta1':0.5,
              'beta2':0.999,
              'batch_size':64,
              'checkpoint_dir': '/content/drive/MyDrive/emojis/checkpoints',
              'sample_dir': '/content/drive/MyDrive/emojis/samples',
              'load': None,
              'log_step':200,
              'sample_every':1000,
              'checkpoint_every':1000,
              'spectral_norm': False,
              'gradient_penalty': False,
              'd_train_iters': 1
}
args = AccessDictByDot.load(args)
train(args)

Models moved to GPU.
Iteration [ 200/10000] | D_real_loss: 0.0007 | D_fake_loss: 0.0024 | G_loss: 0.0145
Iteration [ 400/10000] | D_real_loss: 0.0012 | D_fake_loss: 0.0011 | G_loss: 0.0090
Iteration [ 600/10000] | D_real_loss: 0.0009 | D_fake_loss: 0.0045 | G_loss: 0.0157
Iteration [ 800/10000] | D_real_loss: 0.0019 | D_fake_loss: 0.0005 | G_loss: 0.0090
Iteration [1000/10000] | D_real_loss: 0.0018 | D_fake_loss: 0.0012 | G_loss: 0.0049
Iteration [1200/10000] | D_real_loss: 0.0009 | D_fake_loss: 0.0014 | G_loss: 0.0105
Iteration [1400/10000] | D_real_loss: 0.0009 | D_fake_loss: 0.0018 | G_loss: 0.0312
Iteration [1600/10000] | D_real_loss: 0.0009 | D_fake_loss: 0.0009 | G_loss: 0.0234
Iteration [1800/10000] | D_real_loss: 0.0036 | D_fake_loss: 0.0025 | G_loss: 0.0072
Iteration [2000/10000] | D_real_loss: 0.0002 | D_fake_loss: 0.0007 | G_loss: 0.0088
Iteration [2200/10000] | D_real_loss: 0.0002 | D_fake_loss: 0.0003 | G_loss: 0.0158
Iteration [2400/10000] | D_real_loss: 0.0005 | D_fake_l

In [10]:
from access_dict_by_dot import AccessDictByDot

args = {
              'image_size':32,
              'g_conv_dim':32,
              'd_conv_dim':64,
              'noise_size':100,
              'train_iters':10000,
              'train_path': '/content/drive/MyDrive/Apple/',
              "test_path": '/content/drive/MyDrive/emojis/Test_Apple',
              'lr':0.0005,
              'beta1':0.5,
              'beta2':0.999,
              'batch_size':64,
              'checkpoint_dir': '/content/drive/MyDrive/emojis/checkpoints',
              'sample_dir': '/content/drive/MyDrive/emojis/samples',
              'load': None,
              'log_step':200,
              'sample_every':1000,
              'checkpoint_every':1000,
              'spectral_norm': True,
              'gradient_penalty': False,
              'd_train_iters': 1
}
args = AccessDictByDot.load(args)
train(args)

Models moved to GPU.
Iteration [ 200/10000] | D_real_loss: 0.0037 | D_fake_loss: 0.0141 | G_loss: 0.0649
Iteration [ 400/10000] | D_real_loss: 0.0011 | D_fake_loss: 0.0007 | G_loss: 0.0132
Iteration [ 600/10000] | D_real_loss: 0.0007 | D_fake_loss: 0.0020 | G_loss: 0.0160
Iteration [ 800/10000] | D_real_loss: 0.0014 | D_fake_loss: 0.0003 | G_loss: 0.0118
Iteration [1000/10000] | D_real_loss: 0.0010 | D_fake_loss: 0.0003 | G_loss: 0.0138
Iteration [1200/10000] | D_real_loss: 0.0002 | D_fake_loss: 0.0004 | G_loss: 0.0195
Iteration [1400/10000] | D_real_loss: 0.0006 | D_fake_loss: 0.0003 | G_loss: 0.0142
Iteration [1600/10000] | D_real_loss: 0.0002 | D_fake_loss: 0.0004 | G_loss: 0.0202
Iteration [1800/10000] | D_real_loss: 0.0005 | D_fake_loss: 0.0010 | G_loss: 0.0247
Iteration [2000/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0003 | G_loss: 0.0177
Iteration [2200/10000] | D_real_loss: 0.0006 | D_fake_loss: 0.0002 | G_loss: 0.0157
Iteration [2400/10000] | D_real_loss: 0.0006 | D_fake_l

In [ ]:
from access_dict_by_dot import AccessDictByDot

args = {
              'image_size':32,
              'g_conv_dim':32,
              'd_conv_dim':64,
              'noise_size':100,
              'train_iters':10000,
              'train_path': '/content/drive/MyDrive/Apple/',
              "test_path": '/content/drive/MyDrive/emojis/Test_Apple',
              'lr':0.1,
              'beta1':0.5,
              'beta2':0.999,
              'batch_size':64,
              'checkpoint_dir': '/content/drive/MyDrive/emojis/checkpoints',
              'sample_dir': '/content/drive/MyDrive/emojis/samples',
              'load': None,
              'log_step':200,
              'sample_every':1000,
              'checkpoint_every':1000,
              'spectral_norm': False,
              'gradient_penalty': False,
              'd_train_iters': 1
}
args = AccessDictByDot.load(args)
train(args)

Models moved to GPU.
Iteration [ 200/10000] | D_real_loss: 0.0029 | D_fake_loss: 0.0032 | G_loss: 0.0158
Iteration [ 400/10000] | D_real_loss: 0.0016 | D_fake_loss: 0.0122 | G_loss: 0.0432
Iteration [ 600/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0028 | G_loss: 0.0066
Iteration [ 800/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0003 | G_loss: 0.0123
Iteration [1000/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0020 | G_loss: 0.0305
Iteration [1200/10000] | D_real_loss: 0.0000 | D_fake_loss: 0.0002 | G_loss: 0.0122
Iteration [1400/10000] | D_real_loss: 0.0000 | D_fake_loss: 0.0011 | G_loss: 0.0263
Iteration [1600/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0000 | G_loss: 0.0171
Iteration [1800/10000] | D_real_loss: 0.0000 | D_fake_loss: 0.0006 | G_loss: 0.0230
Iteration [2000/10000] | D_real_loss: 0.0000 | D_fake_loss: 0.0006 | G_loss: 0.0117
Iteration [2200/10000] | D_real_loss: 0.0000 | D_fake_loss: 0.0000 | G_loss: 0.0162
Iteration [2400/10000] | D_real_loss: 0.0000 | D_fake_l

In [9]:
from access_dict_by_dot import AccessDictByDot

args = {
              'image_size':32,
              'g_conv_dim':32,
              'd_conv_dim':64,
              'noise_size':100,
              'train_iters':10000,
              'train_path': '/content/drive/MyDrive/Apple/',
              "test_path": '/content/drive/MyDrive/emojis/Test_Apple',
              'lr':0.01,
              'beta1':0.5,
              'beta2':0.999,
              'batch_size':64,
              'checkpoint_dir': '/content/drive/MyDrive/emojis/checkpoints',
              'sample_dir': '/content/drive/MyDrive/emojis/samples',
              'load': None,
              'log_step':200,
              'sample_every':1000,
              'checkpoint_every':1000,
              'spectral_norm': False,
              'gradient_penalty': False,
              'd_train_iters': 1
}
args = AccessDictByDot.load(args)
train(args)

Models moved to GPU.
Iteration [ 200/10000] | D_real_loss: 0.0392 | D_fake_loss: 0.0070 | G_loss: 0.0092
Iteration [ 400/10000] | D_real_loss: 0.0010 | D_fake_loss: 0.0027 | G_loss: 0.0217
Iteration [ 600/10000] | D_real_loss: 0.0005 | D_fake_loss: 0.0003 | G_loss: 0.0149
Iteration [ 800/10000] | D_real_loss: 0.0000 | D_fake_loss: 0.0001 | G_loss: 0.0153
Iteration [1000/10000] | D_real_loss: 0.0002 | D_fake_loss: 0.0005 | G_loss: 0.0149
Iteration [1200/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0012 | G_loss: 0.0160
Iteration [1400/10000] | D_real_loss: 0.0012 | D_fake_loss: 0.0076 | G_loss: 0.0193
Iteration [1600/10000] | D_real_loss: 0.0027 | D_fake_loss: 0.0005 | G_loss: 0.0063
Iteration [1800/10000] | D_real_loss: 0.0033 | D_fake_loss: 0.0093 | G_loss: 0.0116
Iteration [2000/10000] | D_real_loss: 0.0004 | D_fake_loss: 0.0008 | G_loss: 0.0136
Iteration [2200/10000] | D_real_loss: 0.0003 | D_fake_loss: 0.0001 | G_loss: 0.0137
Iteration [2400/10000] | D_real_loss: 0.0005 | D_fake_l

In [10]:
from access_dict_by_dot import AccessDictByDot

args = {
              'image_size':32,
              'g_conv_dim':32,
              'd_conv_dim':64,
              'noise_size':100,
              'train_iters':10000,
              'train_path': '/content/drive/MyDrive/Apple/',
              "test_path": '/content/drive/MyDrive/emojis/Test_Apple',
              'lr':0.001,
              'beta1':0.5,
              'beta2':0.999,
              'batch_size':64,
              'checkpoint_dir': '/content/drive/MyDrive/emojis/checkpoints',
              'sample_dir': '/content/drive/MyDrive/emojis/samples',
              'load': None,
              'log_step':200,
              'sample_every':1000,
              'checkpoint_every':1000,
              'spectral_norm': False,
              'gradient_penalty': False,
              'd_train_iters': 1
}
args = AccessDictByDot.load(args)
train(args)

Models moved to GPU.
Iteration [ 200/10000] | D_real_loss: 0.0031 | D_fake_loss: 0.0005 | G_loss: 0.0075
Iteration [ 400/10000] | D_real_loss: 0.0022 | D_fake_loss: 0.0048 | G_loss: 0.0118
Iteration [ 600/10000] | D_real_loss: 0.0045 | D_fake_loss: 0.0006 | G_loss: 0.0038
Iteration [ 800/10000] | D_real_loss: 0.0012 | D_fake_loss: 0.0018 | G_loss: 0.0118
Iteration [1000/10000] | D_real_loss: 0.0006 | D_fake_loss: 0.0024 | G_loss: 0.0145
Iteration [1200/10000] | D_real_loss: 0.0007 | D_fake_loss: 0.0004 | G_loss: 0.0185
Iteration [1400/10000] | D_real_loss: 0.0004 | D_fake_loss: 0.0005 | G_loss: 0.0157
Iteration [1600/10000] | D_real_loss: 0.0004 | D_fake_loss: 0.0036 | G_loss: 0.0077
Iteration [1800/10000] | D_real_loss: 0.0023 | D_fake_loss: 0.0031 | G_loss: 0.0305
Iteration [2000/10000] | D_real_loss: 0.0007 | D_fake_loss: 0.0007 | G_loss: 0.0212
Iteration [2200/10000] | D_real_loss: 0.0003 | D_fake_loss: 0.0002 | G_loss: 0.0126
Iteration [2400/10000] | D_real_loss: 0.0013 | D_fake_l

In [ ]:
from access_dict_by_dot import AccessDictByDot

args = {
              'image_size':32,
              'g_conv_dim':32,
              'd_conv_dim':64,
              'noise_size':100,
              'train_iters':10000,
              'train_path': '/content/drive/MyDrive/Apple/',
              "test_path": '/content/drive/MyDrive/emojis/Test_Apple',
              'lr':0.0005,
              'beta1':0.5,
              'beta2':0.999,
              'batch_size':64,
              'checkpoint_dir': '/content/drive/MyDrive/emojis/checkpoints',
              'sample_dir': '/content/drive/MyDrive/emojis/samples',
              'load': None,
              'log_step':200,
              'sample_every':1000,
              'checkpoint_every':1000,
              'spectral_norm': False,
              'gradient_penalty': False,
              'd_train_iters': 2
}
args = AccessDictByDot.load(args)
train(args)

Models moved to GPU.
Iteration [ 200/10000] | D_real_loss: 0.0049 | D_fake_loss: 0.0011 | G_loss: 0.0077
Iteration [ 400/10000] | D_real_loss: 0.0004 | D_fake_loss: 0.0015 | G_loss: 0.0128
Iteration [ 600/10000] | D_real_loss: 0.0010 | D_fake_loss: 0.0013 | G_loss: 0.0074
Iteration [ 800/10000] | D_real_loss: 0.0022 | D_fake_loss: 0.0008 | G_loss: 0.0076
Iteration [1000/10000] | D_real_loss: 0.0015 | D_fake_loss: 0.0003 | G_loss: 0.0144
Iteration [1200/10000] | D_real_loss: 0.0005 | D_fake_loss: 0.0004 | G_loss: 0.0142
Iteration [1400/10000] | D_real_loss: 0.0003 | D_fake_loss: 0.0002 | G_loss: 0.0160
Iteration [1600/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0002 | G_loss: 0.0157
Iteration [1800/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0002 | G_loss: 0.0163
Iteration [2000/10000] | D_real_loss: 0.0004 | D_fake_loss: 0.0003 | G_loss: 0.0149
Iteration [2200/10000] | D_real_loss: 0.0002 | D_fake_loss: 0.0002 | G_loss: 0.0187
Iteration [2400/10000] | D_real_loss: 0.0001 | D_fake_l

In [ ]:
from access_dict_by_dot import AccessDictByDot

args = {
              'image_size':32,
              'g_conv_dim':32,
              'd_conv_dim':64,
              'noise_size':100,
              'train_iters':10000,
              'train_path': '/content/drive/MyDrive/Apple/',
              "test_path": '/content/drive/MyDrive/emojis/Test_Apple',
              'lr':0.0005,
              'beta1':0.5,
              'beta2':0.999,
              'batch_size':64,
              'checkpoint_dir': '/content/drive/MyDrive/emojis/checkpoints',
              'sample_dir': '/content/drive/MyDrive/emojis/samples',
              'load': None,
              'log_step':200,
              'sample_every':1000,
              'checkpoint_every':1000,
              'spectral_norm': False,
              'gradient_penalty': False,
              'd_train_iters': 4
}
args = AccessDictByDot.load(args)
train(args)

Models moved to GPU.
Iteration [ 200/10000] | D_real_loss: 0.0002 | D_fake_loss: 0.0005 | G_loss: 0.0144
Iteration [ 400/10000] | D_real_loss: 0.0006 | D_fake_loss: 0.0005 | G_loss: 0.0122
Iteration [ 600/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0002 | G_loss: 0.0147
Iteration [ 800/10000] | D_real_loss: 0.0018 | D_fake_loss: 0.0023 | G_loss: 0.0069
Iteration [1000/10000] | D_real_loss: 0.0002 | D_fake_loss: 0.0003 | G_loss: 0.0131
Iteration [1200/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0002 | G_loss: 0.0153
Iteration [1400/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0001 | G_loss: 0.0157
Iteration [1600/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0003 | G_loss: 0.0172
Iteration [1800/10000] | D_real_loss: 0.0005 | D_fake_loss: 0.0022 | G_loss: 0.0054
Iteration [2000/10000] | D_real_loss: 0.0001 | D_fake_loss: 0.0002 | G_loss: 0.0164
Iteration [2200/10000] | D_real_loss: 0.0000 | D_fake_loss: 0.0001 | G_loss: 0.0170
Iteration [2400/10000] | D_real_loss: 0.0001 | D_fake_l